In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from disturb_Qavg_r3bp_helio import Q_averaged_disturb
from eq_points_r3bp_helio_conservative import lpe_r3bp_helio_kappa, root_find

$\gamma = \frac{1}{2}\dot{\Gamma}_{\Gamma} + \frac{1}{2}\dot{\kappa}_{\Gamma}\left(\frac{K_{0,\Gamma\kappa}}{K_{0,\Gamma\Gamma}}\right)$
--

$\kappa_2 = \frac{a_2}{a_{2, res}}\left(\frac{j\sqrt{1-e_2^2}-(j-k)}{k}\right)^2 - 1$

$\Gamma_2 = \sqrt{G m_0(1-\beta)a_2}(1-\sqrt{1-e_2^2})$


**step 1 - 1**

From eq (107):
$\dot{\kappa_2} = (1+\kappa_2)\left(\frac{\beta G m_0}{a_2^2 c}\right)\left[-\frac{2+3e_2^2}{(1-e_2^2)^{3/2}}+\frac{5je_2^2}{[j\sqrt{1-e_2^2}-j+k](1-e_2^2)}\right]$

$\dot{\kappa}_{\Gamma} = \frac{\partial \dot{\kappa}}{\partial e_2} \frac{\partial e_2}{\partial \Gamma}$, with $\phi_2$, $\kappa_2$ fixed



In [3]:
beta, G, m0, a2_res, c, j, k = sp.symbols('beta G m0 a2_res c j k')
kappa2, e2 = sp.symbols('kappa2 e2')

# a2(kappa2, e2)
a2 = (kappa2+1)*a2_res / ((j * sp.sqrt(1-e2**2) - (j-k)) / k)**2
# Gamma2(a2(kappa2, e2), e2) = Gamma2(kappa2, e2)
Gamma2 = sp.sqrt(G*m0*(1-beta)*a2)*(1-sp.sqrt(1-e2**2))

Gamma2

sqrt(G*a2_res*k**2*m0*(1 - beta)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(1 - sqrt(1 - e2**2))

In [4]:
# fix kappa2
d_Gamma2_d_e2 = sp.diff(Gamma2, e2)
d_Gamma2_d_e2 = sp.simplify(d_Gamma2_d_e2)
d_e2_d_Gamma2 = 1/d_Gamma2_d_e2
d_e2_d_Gamma2

sqrt(1 - e2**2)*(j*sqrt(1 - e2**2) - j + k)/(e2*sqrt(-G*a2_res*k**2*m0*(beta - 1)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(j*sqrt(1 - e2**2) - j*(sqrt(1 - e2**2) - 1) - j + k))

In [5]:
# kappa2_dot(kappa2, e2)
term1 = (2+3*e2**2) / (1 - e2**2)**(3/2)
term2 = 5 * j * e2**2 / ((j * sp.sqrt(1 - e2**2) - j + k) * (1 - e2**2))
kappa2_dot = (1 + kappa2) * beta*G*m0/(a2**2*c) * (-term1 + term2)

kappa2_dot

G*beta*m0*(5*e2**2*j/((1 - e2**2)*(j*sqrt(1 - e2**2) - j + k)) - (3*e2**2 + 2)/(1 - e2**2)**1.5)*(j*sqrt(1 - e2**2) - j + k)**4/(a2_res**2*c*k**4*(kappa2 + 1))

In [6]:
# fix kappa2
d_kappa2dot_d_e2 = sp.diff(kappa2_dot, e2)
d_kappa2dot_d_e2

-4*G*beta*e2*j*m0*(5*e2**2*j/((1 - e2**2)*(j*sqrt(1 - e2**2) - j + k)) - (3*e2**2 + 2)/(1 - e2**2)**1.5)*(j*sqrt(1 - e2**2) - j + k)**3/(a2_res**2*c*k**4*sqrt(1 - e2**2)*(kappa2 + 1)) + G*beta*m0*(j*sqrt(1 - e2**2) - j + k)**4*(5*e2**3*j**2/((1 - e2**2)**(3/2)*(j*sqrt(1 - e2**2) - j + k)**2) + 10*e2**3*j/((1 - e2**2)**2*(j*sqrt(1 - e2**2) - j + k)) + 10*e2*j/((1 - e2**2)*(j*sqrt(1 - e2**2) - j + k)) - 3.0*e2*(3*e2**2 + 2)/(1 - e2**2)**2.5 - 6*e2/(1 - e2**2)**1.5)/(a2_res**2*c*k**4*(kappa2 + 1))

In [7]:
d_kappa2dot_d_Gamma = d_kappa2dot_d_e2*d_e2_d_Gamma2
d_kappa2dot_d_Gamma = sp.simplify(d_kappa2dot_d_Gamma)
d_kappa2dot_d_Gamma

-G*beta*m0*(4*j*(1 - e2**2)**8.5*(5*e2**2*j*(1 - e2**2)**1.5 + (e2**2 - 1)*(3*e2**2 + 2)*(j*sqrt(1 - e2**2) - j + k)) - (1 - e2**2)**3.0*(5*e2**2*j**2*(1 - e2**2)**7.0 + 10*e2**2*j*(1 - e2**2)**6.5*(j*sqrt(1 - e2**2) - j + k) + 10*j*(1 - e2**2)**7.5*(j*sqrt(1 - e2**2) - j + k) - (1 - e2**2)**6.0*(9.0*e2**2 + 6.0)*(j*sqrt(1 - e2**2) - j + k)**2 - 6*(1 - e2**2)**7.0*(j*sqrt(1 - e2**2) - j + k)**2))*(j*sqrt(1 - e2**2) - j + k)**3/(a2_res**2*c*k**4*sqrt(-G*a2_res*k**2*m0*(beta - 1)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(1 - e2**2)**11.0*(kappa2 + 1)*(j*sqrt(1 - e2**2) - j*(sqrt(1 - e2**2) - 1) - j + k))

In [8]:
G_ = 6.6743e-11
m_Star = 1.99e30 # solar mass in [kg]
R_Sun = 6.957e8 # solar radius in [m]
a1 = 10 * R_Sun
n1_ = np.sqrt(G_*m_Star/(a1**3))
beta_ = 0.1
n2_eq = 9.904864866756742e-06
a2_eq = (G_*m_Star*(1-beta_)/n2_eq**2)**(1/3)
e2_eq = 0.4811868255521742
a2_res_ = a1 * ((2/(2-1))**(2/3)) * (1-beta_)**(1/3)
kappa2_eq = a2_eq/a2_res_ * (2*np.sqrt(1-e2_eq**2)-1)**2-1
Gamma2_eq = np.sqrt(G_*m_Star*(1-beta_)*a2_eq)*(1-np.sqrt(1-e2_eq**2))

test = d_kappa2dot_d_Gamma.subs({
    G: 6.6743e-11,
    c: 3e8,
    beta: 0.1,
    j: 2.,
    k: 1.,
    m0: m_Star,
    a2_res: a2_eq,
    e2: e2_eq,
    kappa2: kappa2_eq})

In [9]:
test

4.17826609258788e-24

**step 1 - 2**

From eq (108):
$\dot{\Gamma}_2 = \left(\frac{\beta G m_0}{a_2^2 c}\right)\sqrt{Gm_0(1-\beta)a_2}\left
[1-\frac{1+3e_2^2/2}{(1-e_2^2)^{3/2}}\right]$

$\dot{\Gamma}_{\Gamma} = \frac{\partial \dot{\Gamma_2}}{\partial e_2} \frac{\partial e_2}{\partial \Gamma_2}$, with $\phi_2$, $\kappa_2$ fixed

In [10]:
# Gamma2_dot(kappa2, e2)
Gamma2_dot = (beta*G*m0/a2**2/c) * sp.sqrt(G*m0*(1-beta)*a2) * (1-(1+3*e2**2/2)/(1-e2**2)**(3/2))
d_Gamma2dot_d_e2 = sp.diff(Gamma2_dot, e2)
d_Gamma2dot_d_e2

-3*G*beta*e2*j*m0*sqrt(G*a2_res*k**2*m0*(1 - beta)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(-(3*e2**2/2 + 1)/(1 - e2**2)**1.5 + 1)*(j*sqrt(1 - e2**2) - j + k)**3/(a2_res**2*c*k**4*sqrt(1 - e2**2)*(kappa2 + 1)**2) + G*beta*m0*sqrt(G*a2_res*k**2*m0*(1 - beta)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(-3.0*e2*(3*e2**2/2 + 1)/(1 - e2**2)**2.5 - 3*e2/(1 - e2**2)**1.5)*(j*sqrt(1 - e2**2) - j + k)**4/(a2_res**2*c*k**4*(kappa2 + 1)**2)

In [11]:
# fix kappa2
d_Gamma2dot_d_Gamma = d_Gamma2dot_d_e2*d_e2_d_Gamma2
d_Gamma2dot_d_Gamma = sp.simplify(d_Gamma2dot_d_Gamma)
d_Gamma2dot_d_Gamma

G*beta*m0*(3*j*(1 - e2**2)**4.0*(3*e2**2 - 2*(1 - e2**2)**1.5 + 2) - 2*(1 - e2**2)**2.0*((1 - e2**2)**1.5*(4.5*e2**2 + 3.0) + 3*(1 - e2**2)**2.5)*(j*sqrt(1 - e2**2) - j + k))*(j*sqrt(1 - e2**2) - j + k)**4/(2*a2_res**2*c*k**4*(1 - e2**2)**5.5*(kappa2 + 1)**2*(j*sqrt(1 - e2**2) - j*(sqrt(1 - e2**2) - 1) - j + k))

In [12]:
test = d_Gamma2dot_d_Gamma.subs({
    G: 6.6743e-11,
    c: 3e8,
    beta: 0.1,
    j: 2.,
    k: 1.,
    m0: m_Star,
    a2_res: a2_eq,
    e2: e2_eq,
    kappa2: kappa2_eq})

test

-8.10646171284396e-10

**step 2**

$K_0 = - \frac{Gm_0(1-\beta)}{2a_2} - n_1\sqrt{Gm_0(1-\beta)a_2(1-e_2^2)}$

$\kappa_2 = \frac{a_2}{a_{2, res}}\left(\frac{j\sqrt{1-e_2^2}-(j-k)}{k}\right)^2 - 1$

$\Gamma_2 = \sqrt{G m_0(1-\beta)a_2}(1-\sqrt{1-e_2^2})$

**step 2 - 1**

$K_{0,\Gamma\Gamma} = \partial \left(\frac{\partial K_0}{\partial e_2}\frac{\partial e_2}{\partial \Gamma_2}\right) / \partial \Gamma = \frac{\partial \left(\frac{\partial K_0}{\partial e_2}\frac{\partial e_2}{\partial \Gamma_2}\right)}{\partial e_2} \frac{\partial e_2}{\partial \Gamma}$

In [13]:
n1 = sp.symbols('n1')

# K0(kappa2, e2)
K0 = - G*m0*(1-beta)/(2*a2) - n1 * sp.sqrt(G*m0*(1-beta)*a2*(1-e2**2))
# fix kappa2
d_K0_d_e2 = sp.diff(K0, e2)

d_K0_d_e2

G*e2*j*m0*(1 - beta)*(j*sqrt(1 - e2**2) - j + k)/(a2_res*k**2*sqrt(1 - e2**2)*(kappa2 + 1)) - n1*sqrt(G*a2_res*k**2*m0*(1 - beta)*(1 - e2**2)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(G*a2_res*e2*j*k**2*m0*(1 - beta)*sqrt(1 - e2**2)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**3 - G*a2_res*e2*k**2*m0*(1 - beta)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(j*sqrt(1 - e2**2) - j + k)**2/(G*a2_res*k**2*m0*(1 - beta)*(1 - e2**2)*(kappa2 + 1))

In [14]:
# fix kappa2
d_K0_d_Gamma = d_K0_d_e2 * d_e2_d_Gamma2
d_K0_d_Gamma

sqrt(1 - e2**2)*(G*e2*j*m0*(1 - beta)*(j*sqrt(1 - e2**2) - j + k)/(a2_res*k**2*sqrt(1 - e2**2)*(kappa2 + 1)) - n1*sqrt(G*a2_res*k**2*m0*(1 - beta)*(1 - e2**2)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(G*a2_res*e2*j*k**2*m0*(1 - beta)*sqrt(1 - e2**2)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**3 - G*a2_res*e2*k**2*m0*(1 - beta)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(j*sqrt(1 - e2**2) - j + k)**2/(G*a2_res*k**2*m0*(1 - beta)*(1 - e2**2)*(kappa2 + 1)))*(j*sqrt(1 - e2**2) - j + k)/(e2*sqrt(-G*a2_res*k**2*m0*(beta - 1)*(kappa2 + 1)/(j*sqrt(1 - e2**2) - j + k)**2)*(j*sqrt(1 - e2**2) - j*(sqrt(1 - e2**2) - 1) - j + k))

$K_{0,\Gamma\Gamma}$

In [15]:
# fix kappa2
d_dK0d_Gamma_d_e2 = sp.diff(d_K0_d_Gamma, e2)
k0_Gamma_Gamma = d_dK0d_Gamma_d_e2 * d_e2_d_Gamma2

**step 2 - 2**

$K_{0,\Gamma\kappa} = \partial \left(\frac{\partial K_0}{\partial e_2}\frac{\partial e_2}{\partial \Gamma_2}\right) / {\partial \kappa_2} = \frac{\partial \left(\frac{\partial K_0}{\partial e_2}\frac{\partial e_2}{\partial \Gamma_2}\right)}{\partial e_2} \frac{\partial e_2}{\partial \kappa_2} = \frac{\partial K_{0,\Gamma}(\kappa_2(a_2, e_2), e_2)}{\partial e_2} \frac{\partial e_2}{\partial \kappa_2} = \frac{\partial K_{0,\Gamma}(a_2(e_2, \kappa_2), e_2)}{\partial e_2} \frac{\partial e_2}{\partial \kappa_2} = \frac{\partial K_{0,\Gamma}(a_2(e_2, \Gamma_2), e_2)}{\partial e_2} \frac{\partial e_2}{\partial \kappa_2} = \frac{\partial K_{0,\Gamma}(\Gamma_2(e_2, a_2), e_2)}{\partial e_2} \frac{\partial e_2}{\partial \kappa_2} = \frac{\partial K_{0,\Gamma}(\Gamma_2, e_2)}{\partial e_2} \frac{1}{{\partial \kappa_2(\Gamma_2, e_2)}/{\partial e_2}}$

In [16]:
# kappa2(a2, e2)
a2_ = sp.symbols('a2_')
kappa2_ = (a2_/a2_res) * ((j * sp.sqrt(1-e2**2) - (j-k)) / k)**2 - 1

# d_K0_d_Gamma(kappa2(a2, e2), e2) to d_K0_d_Gamma(a2_(kappa2, e2), e2)
# substitute kappa2 with a2 & e2
d_K0_d_Gamma_ = d_K0_d_Gamma.subs('kappa2', kappa2_)

# a2(Gamma2, e2)
Gamma2_ = sp.symbols('Gamma2_')
a2_fixGamma = Gamma2_**2/(G*m0*(1-beta)*(1-sp.sqrt(1-e2**2))**2)

# d_K0_d_Gamma(a2_(kappa2, e2), e2) to d_K0_d_Gamma(a2_fixGamma(Gamma2_, e2), e2) to d_K0_d_Gamma(Gamma2_, e2)
# substitute a2 with Gamma2 & e2
d_K0_d_Gamma_fixGamma = d_K0_d_Gamma_.subs('a2_', a2_fixGamma)

# fix Gamma2
d_K0_d_Gamma_d_e2_fixGamma = sp.diff(d_K0_d_Gamma_fixGamma, e2)


In [17]:
d_K0_d_Gamma_

sqrt(1 - e2**2)*(G*e2*j*m0*(1 - beta)/(a2_*sqrt(1 - e2**2)*(j*sqrt(1 - e2**2) - j + k)) - n1*sqrt(G*a2_*m0*(1 - beta)*(1 - e2**2))*(G*a2_*e2*j*m0*(1 - beta)*sqrt(1 - e2**2)/(j*sqrt(1 - e2**2) - j + k) - G*a2_*e2*m0*(1 - beta))/(G*a2_*m0*(1 - beta)*(1 - e2**2)))*(j*sqrt(1 - e2**2) - j + k)/(e2*sqrt(-G*a2_*m0*(beta - 1))*(j*sqrt(1 - e2**2) - j*(sqrt(1 - e2**2) - 1) - j + k))

In [18]:
# kappa2(a2(Gamma2, e2), e2) to kappa2(Gamma2, e2)
kappa2_fixGamma = (a2_fixGamma/a2_res) * ((j * sp.sqrt(1-e2**2) - (j-k)) / k)**2 - 1
# fix Gamma2
d_kappa2_de2_fixGamma = sp.diff(kappa2_fixGamma, e2)
d_e2_d_kappa2_fixGamma = 1/d_kappa2_de2_fixGamma
d_e2_d_kappa2_fixGamma

1/(-2*Gamma2_**2*e2*j*(j*sqrt(1 - e2**2) - j + k)/(G*a2_res*k**2*m0*(1 - beta)*sqrt(1 - e2**2)*(1 - sqrt(1 - e2**2))**2) - 2*Gamma2_**2*e2*(j*sqrt(1 - e2**2) - j + k)**2/(G*a2_res*k**2*m0*(1 - beta)*sqrt(1 - e2**2)*(1 - sqrt(1 - e2**2))**3))

$K_{0,\Gamma\kappa}$

In [19]:
k0_Gamma_kappa = d_K0_d_Gamma_d_e2_fixGamma * d_e2_d_kappa2_fixGamma

$\gamma = \frac{1}{2}\dot{\Gamma}_{\Gamma} + \frac{1}{2}\dot{\kappa}_{\Gamma}\left(\frac{K_{0,\Gamma\kappa}}{K_{0,\Gamma\Gamma}}\right)$
--

In [20]:
gamma = 1/2 * d_Gamma2dot_d_Gamma + 1/2 * d_kappa2dot_d_Gamma * k0_Gamma_kappa/k0_Gamma_Gamma

In [21]:
G_ = 6.6743e-11
m_Star = 1.99e30 # solar mass in [kg]
R_Sun = 6.957e8 # solar radius in [m]
a1 = 10 * R_Sun
n1_ = np.sqrt(G_*m_Star/(a1**3))
beta_ = 0.1
n2_eq = 9.904864866756742e-06
a2_eq = (G_*m_Star*(1-beta_)/n2_eq**2)**(1/3)
e2_eq = 0.4811868255521742
a2_res_ = a1 * ((2/(2-1))**(2/3)) * (1-beta_)**(1/3)
kappa2_eq = a2_eq/a2_res_ * (2*np.sqrt(1-e2_eq**2)-1)**2-1
Gamma2_eq = np.sqrt(G_*m_Star*(1-beta_)*a2_eq)*(1-np.sqrt(1-e2_eq**2))

gamma_subs = gamma.subs({
    G: 6.6743e-11,
    c: 3e8,
    beta: 0.1,
    j: 2.,
    k: 1.,
    m0: m_Star,
    n1: n1_,
    a2_res: a2_eq,
    e2: e2_eq,
    kappa2: kappa2_eq,
    Gamma2_: Gamma2_eq})

In [22]:
print ("growth rate = {}".format(gamma_subs))

growth rate = 3.80847469070978E-10
